## Perform Batch Inference (Predictions) using SageMaker Batch Transform

### Imports 

In [1]:
from sagemaker import get_execution_role
from time import gmtime, strftime
import pandas as pd
import sagemaker
import boto3
import time

### 1. Essentials

In [2]:
BUCKET = 'sagemaker-demo-892313895307'
PREFIX = 'clf'
REGION = 'us-east-1'

In [3]:
batch_input = f's3://{BUCKET}/{PREFIX}/batch_test/'
batch_input

's3://sagemaker-demo-892313895307/clf/batch_test/'

In [4]:
batch_output = f's3://{BUCKET}/{PREFIX}/batch_test_out/'
batch_output

's3://sagemaker-demo-892313895307/clf/batch_test_out/'

In [5]:
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [6]:
TRAINING_JOB_NAME = 'classifier-2020-11-08-22-53-37-824'  # COPY THIS FROM THE CONSOLE
MODEL_NAME = f'clf-xgboost-model-{current_timestamp}'
BATCH_JOB_NAME = f'clf-xgboost-batch-job-{current_timestamp}'

session = boto3.Session()
sagemaker_execution_role = get_execution_role()
sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client('sagemaker', region_name=REGION)
s3_client = boto3.client('s3')

container_uri = sagemaker.image_uris.retrieve(region=session.region_name, 
                                              framework='xgboost', 
                                              version='1.0-1', 
                                              image_scope='training')

### 2. Create a Model object using previously run training job name

In [7]:
info = sagemaker_client.describe_training_job(TrainingJobName=TRAINING_JOB_NAME)
info

{'TrainingJobName': 'classifier-2020-11-08-22-53-37-824',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:training-job/classifier-2020-11-08-22-53-37-824',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-demo-892313895307/clf/model-artifacts/classifier-2020-11-08-22-53-37-824/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'num_round': '100', 'objective': 'binary:logistic'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'train:merror',
    'Regex': '.*\\[[0-9]+\\].*#011train-merror:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[

In [8]:
model_artifact_url = info['ModelArtifacts']['S3ModelArtifacts']
model_artifact_url

's3://sagemaker-demo-892313895307/clf/model-artifacts/classifier-2020-11-08-22-53-37-824/output/model.tar.gz'

In [9]:
primary_container = {
    'Image': container_uri,
    'ModelDataUrl': model_artifact_url
  }

In [10]:
response = sagemaker_client.create_model(
    ModelName=MODEL_NAME,
    ExecutionRoleArn=sagemaker_execution_role,
    PrimaryContainer=primary_container)

In [11]:
response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:892313895307:model/clf-xgboost-model-2020-12-07-21-05-04',
 'ResponseMetadata': {'RequestId': '526d803f-f145-4f31-9ca6-8ecc8513593e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '526d803f-f145-4f31-9ca6-8ecc8513593e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Mon, 07 Dec 2020 21:05:14 GMT'},
  'RetryAttempts': 0}}

### 3. Create a Batch Transformer for Inference

In [12]:
request = {
    "TransformJobName": BATCH_JOB_NAME,
    "ModelName": MODEL_NAME,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {
        "S3OutputPath": batch_output
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": batch_input 
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.m5.xlarge",
            "InstanceCount": 1
    }
}

In [13]:
response = sagemaker_client.create_transform_job(**request)
response

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:892313895307:transform-job/clf-xgboost-batch-job-2020-12-07-21-05-04',
 'ResponseMetadata': {'RequestId': '04e2370d-02de-4424-af3e-02a67eac6606',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '04e2370d-02de-4424-af3e-02a67eac6606',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '118',
   'date': 'Mon, 07 Dec 2020 21:05:20 GMT'},
  'RetryAttempts': 0}}

In [14]:
while(True):
    response = sagemaker_client.describe_transform_job(TransformJobName=BATCH_JOB_NAME)
    status = response['TransformJobStatus']
    if  status == 'Completed':
        print("Transform job ended with status: {}".format(status))
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    print("Transform job is still in status: {}".format(status))    
    time.sleep(30) 

Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job is still in status: InProgress
Transform job ended with status: Completed


### 4. Evaluate Output

In [15]:
key = f'{PREFIX}/batch_test_out/batch_test.csv.out'

In [16]:
obj = s3_client.get_object(Bucket=BUCKET, Key=key)
results_df = pd.read_csv(obj['Body'], names=['Predictions'])

In [17]:
results_df

,Predictions
0,0.110744
1,0.069763
2,0.049868
3,0.509447
4,0.049621
...,...
2482,0.545041
2483,0.049527
2484,0.214841
2485,0.075910
